In [15]:
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate
import skimage.measure as skitm
from astropy.io import fits
import re
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
def gaussian(x, mu=0, sd=1):
    return np.exp(-(((x - mu) / sd) ** 2) / 2) / (np.sqrt(2 * np.pi) * sd)


def gaussian_kernel(size, sigma=1, verbose=False):
    if sigma % 2 != 0:
        sigma += 1
    size = int(size)
    if size % 2 == 0:
        size += 1
    kernel_1D = np.linspace(-size // 2, size // 2, size)
    kernel_1D = gaussian(kernel_1D, 0, sigma)
    # the product of gaussians is proportional to a gaussian
    kernel_2D = np.outer(kernel_1D.T, kernel_1D.T)
    # kernel_2D /= kernel_2D.max()
    kernel_2D /= kernel_2D.sum()
    return kernel_2D


def convolution(image, kernel, average=False, verbose=False):
    if len(image.shape) == 3:
        if verbose:
            print(f"Image Shape : {image.shape}")
            print(f"Found 3 Channels : {image.shape}. Converting to Gray Channel.")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    if verbose:
        print(f"Image Shape : {image.shape}")
        print(f"Kernel Shape : {kernel.shape}")
        plt.imshow(image, cmap="gray")
        plt.title("Original Image")
        plt.show()

    image_row, image_col = image.shape
    kernel_row, kernel_col = kernel.shape

    # output image
    output = np.zeros(image.shape)

    # pad the image (add zeros around the image)
    pad_height = int((kernel_row - 1 * (kernel_row % 2)) / 2)
    pad_width = int((kernel_col - 1 * (kernel_col % 2)) / 2)
    padded_image = np.zeros((image_row + (2 * pad_height), image_col + (2 * pad_width)))
    padded_image[
        pad_height : image_row + pad_height, pad_width : image_col + pad_width
    ] = image

    # convolve the image with the kernel
    for row in range(image_row):
        for col in range(image_col):
            output[row, col] = np.sum(
                kernel * padded_image[row : row + kernel_row, col : col + kernel_col]
            )

    if average:
        output /= kernel_row * kernel_col

    if verbose:
        print("Output Image size : {}".format(output.shape))
        plt.imshow(output, cmap="gray")
        plt.title(f"Output Image using a {kernel_row}X{kernel_col} Kernel")
        plt.show()

    return output

##adjusting resolution between the image and psf
def adjust_resolution(img, old_res=0.05859375, new_res=0.2, func=np.nansum):
    # size = img.shape[-1]
    q = int(new_res / old_res)
    new_img = skitm.block_reduce(img, (q, q), func)
    # Check if the resulting shape is even
    if new_img.shape[0] % 2 == 0:
        new_img = np.vstack([new_img, np.zeros((1, new_img.shape[1]))])
    if new_img.shape[1] % 2 == 0:
        new_img = np.hstack([new_img, np.zeros((new_img.shape[0], 1))])
    return new_img

##PSF convolving function
def PSF_convolve(img, sigma=4, psf=None):
    if psf is None:
        psf = gaussian_kernel(10 * sigma, sigma=sigma)
    img_blur = convolution(img, psf)
    return img_blur, psf

def img_filtering(img_file,sed_file, z):
    c = 2.99e8 #m/s
    lam = np.loadtxt(sed_file, usecols=0)
    with fits.open(img_file) as data:
        img_cube_lambda = data[0].data #units same as surface brightness
        header_cube_disk = data[0].header
        res = header_cube_disk["CDELT1"]
    if 'obs' not in img_file:
        lam *= (1 + z)  ##microns
        img_cube_lambda *= (1 + z)**(-4)
    nu = c / (lam * 1e-6)
    img_cube_nv = (4.255 * 1e24 * lam[:, None, None] * img_cube_lambda / c)  # MJy/sr
    img_disk_nu = np.sum(img_cube_nv,axis=0)
    lam_f, n_f = np.loadtxt(f_filter, unpack=True, skiprows=1)  # micron
    nu_f = c / (lam_f * 1e-6)
    n_interp = np.interp(lam, lam_f, n_f)
    img_cube_nv[np.isnan(img_cube_nv)] = 0
    img_fil = integrate.simpson(img_cube_nv * n_interp[:, None, None], nu, axis=0)
    img_fil /= integrate.simpson(n_interp, nu, axis=0)
    return img_fil, header_cube_disk, res, img_disk_nu, img_cube_lambda

<font size='5'>
Image Filtering

In [9]:
##––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
#INPUT PARAMETERS
#––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
#snap_id = [96]  #z=6.3
snap_id = [66]  #z=8
halo_id = list(range(0,1))
z       = 8.0
##---------------------------------------------------------------------------------------------
#CONFIG PARAMETERS
detection_threshold          = 2.0      #[1:2.5]
background_cell_size         = 64       #[32:256]
smoothing_size               = 7        #[]
core_minimum_area            = 50       #300
detection_minimum_area       = 100      #200
core_threshold_value         = 30       #30
grouping_algorithm           = 'SPLIT'  #[NONE|OVERLAP|SPLIT|MOFFAT] #SPLIT
grouping_moffat_max_distance = 300      #maximum distance between groups  #300

##-----------------------------------------------------------------------------------------
bins = 5
scale = 1
std = scale*np.linspace(0.001, 0.01, bins)  # in the units of img
f0 = '/shares/feldmann.ics.mnf.uzh/data/MassiveFIRE/SKIRT/FIREBOX/FIREBox/FB15N2048/rob'
#––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
#––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
hstr      = []
for halo in halo_id:
    hstr.append(str(halo).rjust(5,'0'))
sstr      = []
for snap in snap_id:
    sstr.append(str(snap).rjust(3,'0'))



for snapstr in sstr:
    for halostr in hstr:
## LOOK FOR THE IMAGE (FACE-ON; TOTAL)
        img_path = f'{f0}/snapshot_{snapstr}/{halostr}/FB15N2048_{snapstr}_{halostr}_XY_total.fits'
        img_files = glob(img_path)
        img_files.sort()
        print(f'Image found :D {img_files}')
## LOOK FOR SED FILE (TO GET THE WAVELENGTHS)
        sed_path = f'{f0}/snapshot_{snapstr}/{halostr}/FB15N2048_{snapstr}_{halostr}_XY_sed.dat'
        sed_files = glob(sed_path)
        sed_files.sort()
        print(f'SED found :D {sed_files}')

f_filter = "/shares/feldmann.ics.mnf.uzh/Haojie/SE++/F356W_mean_system_throughput.txt"
f_psf = "/shares/feldmann.ics.mnf.uzh/Haojie/SE++/PSF_NIRCam_in_flight_opd_filter_F356W.fits"
f_script = "/shares/feldmann.ics.mnf.uzh/Haojie/SE++/template_script_nobackg.py"
f_config = "/shares/feldmann.ics.mnf.uzh/Haojie/SE++/template_configuration.config"

Image found :D ['/shares/feldmann.ics.mnf.uzh/data/MassiveFIRE/SKIRT/FIREBOX/FIREBox/FB15N2048/rob/snapshot_066/00000/FB15N2048_066_00000_XY_total.fits']
SED found :D ['/shares/feldmann.ics.mnf.uzh/data/MassiveFIRE/SKIRT/FIREBOX/FIREBox/FB15N2048/rob/snapshot_066/00000/FB15N2048_066_00000_XY_sed.dat']


In [10]:
##psf upload
with fits.open(f_psf) as data:
    psf = data[1].data
    header_psf = data[1].header
    res_jwst = header_psf["PIXELSCL"]  # [arcseconds/pixel]

##IMAGE RESOLUTION
with fits.open(img_files[0]) as galaxy_file:
    header_cube_disk = galaxy_file[0].header
    res = header_cube_disk["CDELT1"]

<font size='5'>
Background Generation

In [12]:
img_backg = {}
size = (1024, 1024)
mean = 0.00

for i in range(bins):
    if res > res_jwst:
        new_psf         = adjust_resolution(psf, old_res=res_jwst, new_res=res)
        psf_sum         = np.nansum(new_psf)
        psf_squared_sum = np.nansum(new_psf**2)
        std_pre         = std * np.sqrt((psf_sum)**2/psf_squared_sum)
        img_backg[i]    = np.random.normal(mean, std_pre[i], size)
        print(f'img_backg_std_{std_pre[i]:.5f} generated')
        print('size=',size)
    else:
        new_psf         = psf
        psf_sum         = np.nansum(new_psf)
        psf_squared_sum = np.nansum(new_psf**2)
        std_pre         = std * np.sqrt((psf_sum)**2/psf_squared_sum)
        img             = np.random.normal(mean, std_pre[i], size)
        new_size        = adjust_resolution(img, old_res=res, new_res=res_jwst).shape
        img_backg[i]    = np.random.normal(mean, std_pre[i], new_size)
        print(f'img_backg_std_{std_pre[i]:.5f} generated')
        print('size=',new_size)

img_backg_std_0.00131 generated
size= (1024, 1024)
img_backg_std_0.00427 generated
size= (1024, 1024)
img_backg_std_0.00723 generated
size= (1024, 1024)
img_backg_std_0.01018 generated
size= (1024, 1024)
img_backg_std_0.01314 generated
size= (1024, 1024)


<font size='5'>
Operations

In [14]:
##––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––---
#CONSTANTS
c = 2.99e8 #m/s
##––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––---
#operations and directories generation

for snap in sstr:
    for i in range(len(hstr)):
        ##open image
        img_fil, header_cube_disk, res, _, _= img_filtering(img_files[i],sed_files[i],z)
    
        ##creating header for our galaxy
        header_disk = header_cube_disk.copy()
        header_disk["NAXIS"] = 2
        header_disk["BUNIT"] = "MJy/sr"
        del header_disk["CUNIT3"]
        del header_disk["NAXIS3"]
        
    
        ##add the background to the filtered image
        img_fil_backgadd = {}
        img_fil_backgadd_psf = {}
        kernel = {}
        for j in range(bins):
            img_fil_backgadd[j] = img_fil + img_backg[j]
    
        ##psf convolving
            if res > res_jwst:
                print("image res. > JWST PSF res. ---> degrading psf resolution")
                print(f"convolving img_fil_backgadd_std_{std[j]:.5f}")
                new_psf = adjust_resolution(psf, old_res=res_jwst, new_res=res)
                img_fil_backgadd_psf[j], kernel[j] = PSF_convolve(img_fil_backgadd[j], psf=new_psf)
                header_new_psf = header_psf.copy()
                header_new_psf["NAXIS1"] = kernel[j].shape[0]
                header_new_psf["NAXIS2"] = kernel[j].shape[1]
                header_new_psf["PIXELSCL"] = res
            else:
                print("image res. < JWST PSF res. ---> degrading image resolution")
                new_img_fil = adjust_resolution(img_fil_backgadd[j], old_res=res, new_res=res_jwst)
                img_fil_backgadd_psf[j], kernel[j] = PSF_convolve(new_img_fil, psf=psf)
                header_new_psf = header_psf.copy()
                header_new_psf["NAXIS1"] = kernel[j].shape[0]
                header_new_psf["NAXIS2"] = kernel[j].shape[1]
                header_new_psf["PIXELSCL"] = res_jwst
    
            ##creating the directory      
            directory_path = f"/shares/feldmann.ics.mnf.uzh/Haojie/SE++/FB15N2048/snap_{snap}/halo_{hstr[i]}/std_{std[j]:.5f}"
            os.makedirs(directory_path, exist_ok=True)
            print(f"Directory {directory_path} created successfully")
                
            ##saving background image
            hdu_backg = fits.PrimaryHDU(img_backg[j], header_disk)
            hdulist_backg = fits.HDUList([hdu_backg])
            output_filename_backg = (f"{directory_path}/backg_snap_{snap}_halo_{hstr[i]}_std_{std_pre[j]:.5f}->{std[j]:.5f}.fits")
            hdulist_backg.writeto(output_filename_backg, overwrite=True)
            print(f"File written: {output_filename_backg}")

            ##saving new_psf image
            hdu = fits.PrimaryHDU(kernel[j],header_new_psf)
            hdulist = fits.HDUList([hdu])
            output_filename_psf = f"{directory_path}/psf_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits"
            hdulist.writeto(output_filename_psf, overwrite=True)
            print(f"File written: {output_filename_psf}")

    
            ##saving filtered image
            hdu_image = fits.PrimaryHDU(img_fil_backgadd_psf[j], header_disk)
            hdulist_image = fits.HDUList([hdu_image])
            image_filename = (f"{directory_path}/image_filtered_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits")
            hdulist_image.writeto(image_filename, overwrite=True)
            print(f"File written: {image_filename}")

            ##creating and write the new script file
            new_script_path = f"{directory_path}/script_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.py"
            new_lines_script = []
            with open(f_script) as file:
                lines_script = file.readlines()
                for line_script in lines_script:
                    if line_script.strip().startswith('image_file ='):
                        new_lines_script.append(f"image_file = '{image_filename}'\n")
                    elif line_script.strip().startswith('psf_file   ='):
                        new_lines_script.append(f"psf_file   = '{output_filename_psf}'\n")
                    else:
                        new_lines_script.append(line_script)
            with open(new_script_path, 'w') as file:
                file.writelines(new_lines_script)
            print(f"Modified script has been saved to '{new_script_path}'.")

            ##creating and write the new configuration file for running SE++
            new_config_path = f"{directory_path}/config_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.config"
            new_config_paths = {
            'check-image-model-fitting=':f'{directory_path}/model_sersic.fits',
            'check-image-residual=':f'{directory_path}/resid_sersic.fits',
            'check-image-background=':f'{directory_path}/background_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits',
            'check-image-variance=':f'{directory_path}/variance_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits',
            'check-image-segmentation=':f'{directory_path}/segmentation_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits',
            'check-image-partition=':f'{directory_path}/partition_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits',
            'check-image-grouping=':f'{directory_path}/grouping_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits',
            'check-image-filtered=':f'{directory_path}/check_filtered_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits',
            'check-image-thresholded=':f'{directory_path}/thresholded_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits',
            'check-image-auto-aperture=':f'{directory_path}/auto_aperture_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits',
            'check-image-psf=':f'{directory_path}/psf_check.fits',
            '#background-value=':'background-value=0.0',
            'detection-image=':image_filename,
            'detection-threshold=':detection_threshold,
            'background-cell-size=':background_cell_size,
            'smoothing-box-size=':smoothing_size,
            'core-minimum-area=':core_minimum_area,
            'detection-minimum-area=':detection_minimum_area,
            'core-threshold-value=':core_threshold_value,
            'grouping-algorithm=':grouping_algorithm,
            'grouping-moffat-max-distance=':grouping_moffat_max_distance,
            'python-config-file=':f"{directory_path}/script_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.py",
            'output-catalog-filename=':f'{directory_path}/catalog_snap_{snap}_halo_{hstr[i]}_std_{std[j]:.5f}.fits',
            'psf-filename=':f'{output_filename_psf}',
            'log-file=':f'{directory_path}/snap_{snap}_halo_{hstr[i]}std_{std[j]:.5f}.log'
            }
            new_lines_config = []
            with open(f_config) as file:
                lines_config = file.readlines()
            for line_config in lines_config:
                modified = False
                for key_word,new_check_image_path in new_config_paths.items():
                    if line_config.strip().startswith(key_word):
                        if key_word == '#background-value=':
                            new_line_config = f"{new_check_image_path}\n"
                        else:
                            new_line_config = f"{key_word}{new_check_image_path}\n"
                        new_lines_config.append(new_line_config)
                        modified = True
                        break
                if not modified:
                    new_lines_config.append(line_config)
            with open(new_config_path, 'w') as file:
                file.writelines(new_lines_config)
            print(f"Modified configuration has been saved to '{new_config_path}'.")

/sctmp/hgeng/ipykernel_2284303/2795902781.py:111: DeprecationWarning: You are passing x=[3.32222222e+15 3.21471861e+15 3.11069370e+15 3.01003493e+15
 2.91263338e+15 2.81838363e+15 2.72718370e+15 2.63893491e+15
 2.55354176e+15 2.47091184e+15 2.39095574e+15 2.31358693e+15
 2.23872169e+15 2.16627902e+15 2.09618051e+15 2.02835032e+15
 1.96271504e+15 1.89920365e+15 1.83774742e+15 1.77827985e+15
 1.72073659e+15 1.66505537e+15 1.61117593e+15 1.55903997e+15
 1.50859108e+15 1.45977466e+15 1.41253789e+15 1.36682966e+15
 1.32260049e+15 1.27980253e+15 1.23838947e+15 1.19831649e+15
 1.15954023e+15 1.12201873e+15 1.08571139e+15 1.05057891e+15
 1.01658329e+15 9.83687721e+14 9.51856623e+14 9.21055546e+14
 8.91251159e+14 8.62411210e+14 8.34504492e+14 8.07500806e+14
 7.81370931e+14 7.56086591e+14 7.31620426e+14 7.07945960e+14
 6.85037574e+14 6.62870480e+14 6.41420690e+14 6.20664992e+14
 6.00580927e+14 5.81146761e+14 5.62341464e+14 5.44144687e+14
 5.26536738e+14 5.09498564e+14 4.93011728e+14 4.77058388e+

image res. > JWST PSF res. ---> degrading psf resolution
convolving img_fil_backgadd_std_0.00100
Directory /shares/feldmann.ics.mnf.uzh/Haojie/SE++/FB15N2048/snap_066/halo_00000/std_0.00100 created successfully
File written: /shares/feldmann.ics.mnf.uzh/Haojie/SE++/FB15N2048/snap_066/halo_00000/std_0.00100/backg_snap_066_halo_00000_std_0.00131->0.00100.fits
File written: /shares/feldmann.ics.mnf.uzh/Haojie/SE++/FB15N2048/snap_066/halo_00000/std_0.00100/psf_snap_066_halo_00000_std_0.00100.fits
File written: /shares/feldmann.ics.mnf.uzh/Haojie/SE++/FB15N2048/snap_066/halo_00000/std_0.00100/image_filtered_snap_066_halo_00000_std_0.00100.fits
Modified script has been saved to '/shares/feldmann.ics.mnf.uzh/Haojie/SE++/FB15N2048/snap_066/halo_00000/std_0.00100/script_snap_066_halo_00000_std_0.00100.py'.
Modified configuration has been saved to '/shares/feldmann.ics.mnf.uzh/Haojie/SE++/FB15N2048/snap_066/halo_00000/std_0.00100/config_snap_066_halo_00000_std_0.00100.config'.
image res. > JWST 